# Characterize potential i-band photometry issue

**UPDATE TUE FEB 27** With the new re-processed photometry, the issue is gone.



**PREVIOUSLY**

Midori noticed that a few of our long-term variables showed a "jump" in their i-band difference-image magnitudes around MJD = 59500. This notebook reproduces the behaviour and tries to quantify how many candidates are impacted.

**Takeaway:** We do find 22 candidates (out of the 8998 being published in the paper) exhibit a ~1 mag "jump" in the i-band around MJD-59500. They're all in ELAIS and all appear to be stellar.

**Question:** What caused this and is it something we can fix before publishing?

## Set up

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
import pandas

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
from astropy.time import Time

from copy import deepcopy
import time

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")
db.autocommit = True
cursor = db.cursor(cursor_factory = psycopg2.extras.DictCursor)

In [ ]:
# tag = "latest"
tag = "default"  # as of Tue Feb 27 2024
db.rollback()
q = ( "SELECT id, tag FROM versiontags WHERE tag=%(tag)s" )
cursor.execute( q, { "tag": tag } )
row = cursor.fetchone()
tagid = row['id']
print(tagid)

In [ ]:
del dbuser, dbpasswd, db, tag, q, row

## Four example candidates

Four long-term variable candidates for which the i-band "jump" was noticed.

In [ ]:
ex_cands = np.asarray(['DC21koqo','DC21kmdi','DC21ksgv','DC21kofv'], dtype='str')

### Retrive difference-image object photometry

In [ ]:
%%time

tmp_str = "('DC21koqo', 'DC21kmdi', 'DC21ksgv', 'DC21kofv')"

q = ("SELECT od.ra, od.dec, od.mag, od.magerr, rbs.rb, "
     "i.filter, i.meanmjd, o.candidate_id, e.proposalid "
     "FROM objectdatas AS od "
     "INNER JOIN objectdata_versiontag AS odvt "
     "ON od.id=odvt.objectdata_id AND odvt.versiontag_id=%(tagid)s "
     "INNER JOIN objects AS o ON od.object_id=o.id "
     "INNER JOIN images AS i ON o.image_id=i.id "
     "INNER JOIN objectrbs as rbs ON od.id=rbs.objectdata_id AND rbs.rbtype_id=2 "
     "INNER JOIN exposures AS e ON i.exposure_id=e.id "
     "WHERE o.candidate_id IN " + tmp_str)
     # "WHERE ((od.ra > 147.0 AND od.ra < 153.0 AND od.dec > -0.25 AND od.dec < 5) "
     # "OR (od.ra > 5.0 AND od.ra < 12.0 AND od.dec > -46 AND od.dec < -41)) ")

cursor.execute(q, {'tagid': tagid})

del tmp_str, q, tagid
     
df = pandas.DataFrame(cursor.fetchall())

print(len(df))

In [ ]:
obj_ra     = np.asarray(df[0], dtype='float')
obj_dec    = np.asarray(df[1], dtype='float')
obj_mag    = np.asarray(df[2], dtype='float')
obj_mage   = np.asarray(df[3], dtype='float')
obj_rb     = np.asarray(df[4], dtype='float')
obj_filt   = np.asarray(df[5], dtype='str')
obj_mjd    = np.asarray(df[6], dtype='float')
obj_candid = np.asarray(df[7], dtype='str')
obj_propid = np.asarray(df[8], dtype='str')
del df

### Plot the light curves

In [ ]:
fnames = ['g','r','i']
fcolors = ['green','red','blue']
fshapes = ['o','s','*']
fsizes  = [5, 5, 7]

In [ ]:
fig, ax = plt.subplots(4, figsize=(6, 6), sharex=False, sharey=False)
for c, cand in enumerate(ex_cands):
    for f, filt in enumerate(fnames):
        tx = np.where((obj_candid == cand) & (obj_filt == filt))[0]
        ax[c].plot(obj_mjd[tx], obj_mag[tx], fshapes[f], ms=fsizes[f], 
                   alpha=0.5, mew=0, color=fcolors[f])
    ax[c].set_title(cand)
    ax[c].invert_yaxis()
fig.tight_layout()
fig.show()

In [ ]:
fig, ax = plt.subplots(4, figsize=(6, 6), sharex=False, sharey=False)
for c, cand in enumerate(ex_cands):
    for f, filt in enumerate(fnames):
        tx = np.where((obj_candid == cand) & (obj_filt == filt))[0]
        ax[c].plot(obj_mjd[tx], obj_mag[tx], fshapes[f], ms=fsizes[f], 
                   alpha=0.5, mew=0, color=fcolors[f])
    ax[c].set_xlim([59480, 59520])
    ax[c].set_title(cand)
    ax[c].invert_yaxis()
fig.tight_layout()
fig.show()

In [ ]:
del cursor
del obj_ra, obj_dec, obj_mag, obj_mage
del obj_rb, obj_filt, obj_mjd, obj_candid, obj_propid
del fig

## Quantify impact

For the paper we're using the 8998 candidates identified as 'likely real'. 

Look for this jump in the nightly-epoch light curves.

Find out how many candidates being used for the paper might be affected.

### Read nightly-epoch light curves

In [ ]:
fnm = '../all_nightly_epochs_files/candidate_lightcurves.dat'
candLC_field = np.loadtxt(fnm, dtype='str', usecols=(0))
candLC_candid = np.loadtxt(fnm, dtype='str', usecols=(1))
candLC_caldate = np.loadtxt(fnm, dtype='str', usecols=(2))
candLC_mjd = np.loadtxt(fnm, dtype='float', usecols=(3))
candLC_filter = np.loadtxt(fnm, dtype='str', usecols=(4))
candLC_Nobj = np.loadtxt(fnm, dtype='int', usecols=(5))
candLC_mag = np.loadtxt(fnm, dtype='float', usecols=(6))
candLC_mage = np.loadtxt(fnm, dtype='float', usecols=(7))
candLC_mrb = np.loadtxt(fnm, dtype='float', usecols=(8))
candLC_lef = np.loadtxt(fnm, dtype='float', usecols=(9))
del fnm

### Plot again for the four examples

In [ ]:
fig, ax = plt.subplots(4, figsize=(6, 6), sharex=False, sharey=False)
for c, cand in enumerate(ex_cands):
    for f, filt in enumerate(fnames):
        tx = np.where((candLC_candid == cand) & (candLC_filter == filt))[0]
        ax[c].plot(candLC_mjd[tx], candLC_mag[tx], fshapes[f], ms=fsizes[f], 
                   alpha=0.8, mew=0, color=fcolors[f])
    ax[c].set_xlim([59480, 59520])
    ax[c].set_title(cand)
    ax[c].invert_yaxis()
fig.tight_layout()
fig.show()

### Number of candidates with a "jump" around MJD 59500 

First, the number of unique candidates. Will be 8998.

In [ ]:
ucands = np.unique(candLC_candid)
print(len(ucands))

Store the mean magnitude in the days before/after MJD 59500 as m1 and m2.

In [ ]:
%%time
m1 = np.zeros(len(ucands), dtype='float')
m2 = np.zeros(len(ucands), dtype='float')
for u, ucand in enumerate(ucands):
    tx = np.where((candLC_candid == ucand) & (candLC_filter == 'i'))[0]
    x1 = np.where((candLC_mjd[tx] >= 59480) & (candLC_mjd[tx] < 59500))[0]
    x2 = np.where((candLC_mjd[tx] > 59500) & (candLC_mjd[tx] <= 59515))[0]
    if (len(x1) > 0) & (len(x2) > 0):
        m1[u] = np.mean(candLC_mag[tx[x1]])
        m2[u] = np.mean(candLC_mag[tx[x2]])
    del tx, x1, x2

Number of candidates that have data before/after MJD 59500.

In [ ]:
tx = np.where(m1 > 0)[0]
print(len(tx))
del tx

Calculate the difference, before and after MJD 59500.

In [ ]:
diff = m2 - m1

Plot the distribution of non-zero differences.

In [ ]:
fig = plt.figure(figsize=(3, 2))
tx = np.where(diff != 0.0)[0]
plt.hist(diff[tx])
plt.xlabel('diff')
plt.show()
del tx

Identify candidates with a 1-mag jump like the candidates that Midori found.

In [ ]:
tx = np.where(diff > 1)[0]
print(len(tx))
del tx

**AS OF TUE FEB 27** there are none!! Previously there had been 22.

## Previous code to explore the affected candidates

Were all four of the examples re-discovered?

In [ ]:
tx = np.where(diff > 1)[0]
count = 0
for ecand in ex_cands:
    ex = np.where(ucands[tx] == ecand)[0]
    count += len(ex)
    del ex
print(count)
del tx, count

Make a list of the 22 "affected candidates".

In [ ]:
tx = np.where(diff > 1)[0]
af_cands = ucands[tx]
del tx

### Plot light curves for the 22 candidates with i-band "jumps" >1 mag

In [ ]:
nrows = 11
ncols = 2
fig, ax = plt.subplots(nrows, ncols, figsize=(10, 20), sharex=False, sharey=False)
c = 0
for i in range(nrows):
    for j in range(ncols):
        cand = af_cands[c]
        ax[i, j].axvline(59500, color='lightgrey')
        for f, filt in enumerate(fnames):
            tx = np.where((candLC_candid == cand) & (candLC_filter == filt))[0]
            ax[i, j].plot(candLC_mjd[tx], candLC_mag[tx], fshapes[f], ms=fsizes[f], 
                       alpha=0.8, mew=0, color=fcolors[f])
            del tx
        # ax[i, j].set_xlim([59480, 59520])
        ax[i, j].set_title(cand)
        ax[i, j].invert_yaxis()
        del cand
        c += 1
fig.tight_layout()
fig.show()
del nrows, ncols, c

### Check if they're all in the same area in ELAIS

In [ ]:
fnm = '../all_nightly_epochs_files/candidate_coordinates.dat'
cc_candid = np.loadtxt(fnm, dtype='str', usecols=(0))
cc_ra = np.loadtxt(fnm, dtype='float', usecols=(1))
cc_dec = np.loadtxt(fnm, dtype='float', usecols=(2))
del fnm

In [ ]:
fig = plt.figure(figsize=(3, 3))

tx = np.where((cc_ra > 5.0) & (cc_ra < 12.0) & 
              (cc_dec > -46) & (cc_dec < -41))[0]
plt.plot(cc_ra[tx], cc_dec[tx], 'o', ms=2,
         alpha=0.2, mew=0, color='lightblue')
del tx

tx = np.where(diff != 0.0)[0]
tmp_cands = ucands[tx]
del tx
for cand in tmp_cands:
    tx = np.where(cc_candid == cand)[0]
    plt.plot(cc_ra[tx[0]], cc_dec[tx[0]], 'o', ms=3, 
             alpha=0.4, mew=0, color='darkblue')
    del tx
del tmp_cands

for cand in af_cands:
    tx = np.where(cc_candid == cand)[0]
    plt.plot(cc_ra[tx[0]], cc_dec[tx[0]], 'o', ms=3,
             alpha=0.8, mew=0, color='red')
    del tx

plt.show()

No they're scattered across the region.

### Check if the 22 candidates are cross-matched with stars or galaxies

In [ ]:
fnm = '../cross_matched_LSDR10/candidate_xmatch_LSdr10.dat'
lsdr10xm_candid   = np.loadtxt(fnm, dtype='str', usecols=(0))
lsdr10xm_flagstar = np.loadtxt(fnm, dtype='int', usecols=(4))
lsdr10xm_flagcore = np.loadtxt(fnm, dtype='int', usecols=(5))
lsdr10xm_flagmatch= np.loadtxt(fnm, dtype='int', usecols=(6))
lsdr10xm_rmag     = np.loadtxt(fnm, dtype='float', usecols=(13))
del fnm

In [ ]:
for cand in af_cands:
    tx = np.where(lsdr10xm_candid == cand)[0]
    tmp1 = '      '
    tmp2 = '    '
    if lsdr10xm_flagstar[tx[0]] == 1:
        tmp1 = 'star'
    elif lsdr10xm_flagstar[tx[0]] == 0:
        tmp1 = 'galaxy'
        if lsdr10xm_flagcore[tx[0]] == 1:
            tmp2 = 'core'
    print('%12s %10s %5s %5.2f' % (cand, tmp1, tmp2, lsdr10xm_rmag[tx[0]]))
    del tx, tmp1, tmp2

### Print the links for follow-up

Links to the Legacy Survey stamp and to Rob Knop's candidate viewer pages.

By checking the LS stamps, looks like even the two that were cross-matched
to "galaxies" were probably stars.

In [ ]:
fnm = '../cross_matched_LSDR10/candidate_links.dat'
link_candid  = np.loadtxt(fnm, dtype='str', usecols=(0))
link_LSstamp = np.loadtxt(fnm, dtype='str', usecols=(1))
link_RKviewer = np.loadtxt(fnm, dtype='str', usecols=(2))
del fnm

In [ ]:
for cand in af_cands:
    tx = np.where(link_candid == cand)[0]
    print(cand)
    print(link_LSstamp[tx[0]])
    print(link_RKviewer[tx[0]])
    del tx